In [ ]:
%pip install tensorflow


In [2]:
%pip install numpy
%pip install matplotlib
%pip install scikit-learn
%pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.


# Data loading and Pre- Processing

In [4]:
# Housekeeping
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
#defining the dataset path and directories
base_path = 'dataset/chest_xray'
dir_train = os.path.join(base_path, 'train')
dir_val = os.path.join(base_path, 'val')
dir_test = os.path.join(base_path, 'test')


In [6]:
# Now, defining our image data generator with preprocessing

#augumentation
train_dataGen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,
    zoom_range=0.2, horizontal_flip= True)


# No Augumentation for test
test_dataGen = ImageDataGenerator(rescale=1./255)

#loading imgages from the directories
train_Gen = train_dataGen.flow_from_directory(dir_train, target_size=(150,150), batch_size=20, class_mode='binary')

valid_Gen = test_dataGen.flow_from_directory(dir_val, target_size=(150,150), batch_size=20, class_mode='binary')

test_Gen = test_dataGen.flow_from_directory(dir_test, target_size=(150,150), batch_size=20, class_mode='binary')



Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
#GoogleNet

In [8]:
import ssl
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
ssl._create_default_https_context = ssl._create_unverified_context

In [9]:
def create_inception_model():
    base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(150, 150, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)  # Example layer
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [10]:
inception_model = create_inception_model()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 23s 0us/step


In [17]:

# Example for GoogleLeNet
history_inception = inception_model.fit(train_Gen, epochs=10, validation_data=valid_Gen)
